In [18]:
# import libs
import cv2
import mediapipe as mp

from skimage.draw import line


In [19]:
# pose estimation utils
mp_pose = mp.solutions.pose

# joint's coordinate diff b/w frames
coord_diff = [0] * 33
# joint's coordinate @ prev frame
prev_coord = [[0, 0] for _ in range(33)]


In [20]:
def calc_coord(lm, coord_diff, prev_coord):
    """Loop through each joint, calculate coordinate difference between frames

    Args:
        lm (NormalizedLandmarks): list of normalized landmarks predicted
        coord_diff (list[float]): list of joint coordinate difference
        prev_coord (list[list[float]]): list of joint's coordinate list (x,y only)
    """
    amplification_factor = 10

    for i in range(33):
        # NOTE: MANHATTAN DISTANCE is used instead of EUCLIDEAN DISTANCE
        coord_diff[i] = min(1, (abs(prev_coord[i][0] - lm[i].x) +
                                abs(prev_coord[i][1] - lm[i].y)) * amplification_factor)
        prev_coord[i] = [lm[i].x, lm[i].y]


In [21]:
def visualize_skeleton(img, lm, line_size, height, width, clr):
    """Visualize default skeleton

    Args:
        img (ndarray): numpy array of frame
        lm (NormalizedLandmarks): list of normalized landmarks predicted
        line_size (int): skeleton line size
        height (int): height of frame 
        width (int): width of frame
        clr (tuple[int])): tuple of RGB values 
    """
    # visualize skeleton for each joint-to-joint connections
    for item in mp_pose.POSE_CONNECTIONS:
        img = cv2.line(img, (int(lm[item[0]].x * width), int(lm[item[0]].y * height)),
                       (int(lm[item[1]].x * width), int(lm[item[1]].y * height)), clr, line_size)


In [22]:
def draw_hm_skeleton(img, lm, coord_diff, prev_coord, height, width, line_size):
    """Draw skeleton with joint activity measure

    Args:
        img (ndarray): numpy array of frame
        lm (NormalizedLandmarks): list of normalized landmarks predicted
        coord_diff (list[float]): list of joint coordinate difference
        prev_coord (list[list[float]]): list of joint's coordinate list (x,y only)
        height (int): height of frame 
        width (int): width of frame
        line_size (int): skeleton line size

    Returns:
        ndarray: numpy array of modified frame
    """

    calc_coord(lm, coord_diff, prev_coord)

    # uncomment this line if you need to render default white skeleton also
    # visualize_skeleton(img, lm, 20, height, width, (255,255,255))

    max_b = 50
    b_spread = 155

    max_g = 50
    g_spread = 155

    max_r = 255
    r_spread = 50

    # for each joint-to-joint connection / line, visualize activity
    for item in mp_pose.POSE_CONNECTIONS:
        # line iterator -scikit image
        lines = list(zip(*line(int(lm[item[0]].x * width),
                               int(lm[item[0]].y * height),
                               int(lm[item[1]].x * width),
                               int(lm[item[1]].y * height))))

        # 1st joint in a connection
        for i in range(0, int(len(lines) * coord_diff[item[0]])-1):
            img = cv2.line(img,
                           (int(lines[i][0]), int(lines[i][1])),
                           (int(lines[i+1][0]), int(lines[i+1][1])),
                           (max_b + int((1 - coord_diff[item[0]]) * b_spread) + i,
                            max_g +
                            int((1 - coord_diff[item[0]]) * g_spread) + i,
                            max_r - int(coord_diff[0] * r_spread)),
                           line_size)

        # 2nd joint in a connection
        for i in range(len(lines) - 1, len(lines) - int(len(lines) * coord_diff[item[1]]), -1):
            img = cv2.line(img,
                           (int(lines[i][0]), int(lines[i][1])),
                           (int(lines[i-1][0]), int(lines[i-1][1])),
                           (max_b + int((1 - coord_diff[item[1]]) * b_spread) + i,
                            max_g +
                            int((1 - coord_diff[item[1]]) * g_spread) + i,
                               max_r - int(coord_diff[1] * r_spread)),
                           line_size)

    return img


In [24]:

# capture
cap = cv2.VideoCapture('sample.mp4')

# init output
out = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(
    *'mp4v'), 24, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))


with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    # while vid file is available and opened or webcam is available and working
    while cap.isOpened():
        ret, frame = cap.read()

        if ret:
            # openCV uses BGR color format but Mediapipe uses RGB
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img.flags.writeable = False

            results = pose.process(img)

            img.flags.writeable = True
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

            # try operations only if landmarks were found
            try:
                lm = results.pose_landmarks.landmark    # landmarks
                img = draw_hm_skeleton(
                    img, lm, coord_diff, prev_coord, frame.shape[0], frame.shape[1], 20)
            except:
                pass

            # feed
            cv2.imshow('feed', img)
            out.write(img)

            # break feed
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        else:
            break

    # release feed
    cap.release()
    out.release()
    cv2.destroyAllWindows()
